In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from entsoe import EntsoePandasClient

In [2]:
my_api_key = os.environ.get('ENTSOE_API_KEY')
client = EntsoePandasClient(api_key=my_api_key)

# Parameters of the dataset

In [3]:
country_code = 'HU'
years = [('2017-01-01', '2017-12-31'),
         ('2018-01-01', '2018-12-31'),
         ('2019-01-01', '2019-12-31'),
         ('2020-01-01', '2020-12-31'),
         ('2021-01-01', '2021-12-31'),
         ('2022-01-01', '2022-12-31'),
         ('2023-01-01', '2023-12-31'),
         ('2024-01-01', '2024-03-24')]

In [4]:
prices_folder_path = './data/prices'
loads_folder_path = './data/loads'
wind_solar_forecast_folder_path = './data/wind_solar_forecast'

In [5]:
def get_base_df_filename(country_code, years):
    return f'./data/base_price_{country_code}_{years[0]}_{years[-1]}.csv'

def get_base_load_df_filename(country_code, years):
    return f'./data/base_load_avgs_{country_code}_{years[0]}_{years[-1]}.csv'

def get_base_wind_solar_forecast_df_filename(country_code, years):
    return f'./data/base_wind_solar_forecast_avgs_{country_code}_{years[0]}_{years[-1]}.csv'

In [6]:
def run_querry_day_ahead_prices(country_code, start_date, end_date):
    filename = f'price_{start_date}_{end_date}_{country_code}.csv'
    start_ts = pd.Timestamp(start_date, tz='Europe/Budapest')
    end_ts = pd.Timestamp(end_date, tz='Europe/Budapest')

    if os.path.exists(f'{prices_folder_path}/{filename}'):
        print(f'{prices_folder_path}/{filename} exists, reading from file')
        #load
        df = pd.read_csv(f'{prices_folder_path}/{filename}', index_col=0)
    else:
        print(f'{prices_folder_path}/{filename} does not exist, downloading from ENTSO-E')

        #set start time to 00:00:00 and end time to 23:59:59
        start_ts = pd.Timestamp(start_date, tz='Europe/Brussels')
        end_ts = pd.Timestamp(end_date, tz='Europe/Brussels') + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)

        df = client.query_day_ahead_prices(country_code, start=start_ts, end=end_ts)        # Data from ENTSO-E
        
        df.to_csv(f'{prices_folder_path}/{filename}')

    return df

In [7]:
def run_querry_load(country_code, start_date, end_date):
    filename = f'load_{start_date}_{end_date}_{country_code}.csv'
    start_ts = pd.Timestamp(start_date, tz='Europe/Budapest')
    end_ts = pd.Timestamp(end_date, tz='Europe/Budapest')

    if os.path.exists(f'{loads_folder_path}/{filename}'):
        print(f'{loads_folder_path}/{filename} exists, reading from file')
        #load
        df = pd.read_csv(f'{loads_folder_path}/{filename}', index_col=0)
    else:
        print(f'{loads_folder_path}/{filename} does not exist, downloading from ENTSO-E')

        #set start time to 00:00:00 and end time to 23:59:59
        start_ts = pd.Timestamp(start_date, tz='Europe/Brussels')
        end_ts = pd.Timestamp(end_date, tz='Europe/Brussels') + pd.Timedelta(days=1)

        df =client.query_load(country_code, start=start_ts, end=end_ts)        # Data from ENTSO-E
        
        df.to_csv(f'{loads_folder_path}/{filename}')

    return df

In [8]:
def run_querry_wind_solar_forecast(country_code, start_date, end_date):
    filename = f'wind_solar_forecast_{start_date}_{end_date}_{country_code}.csv'
    start_ts = pd.Timestamp(start_date, tz='Europe/Budapest')
    end_ts = pd.Timestamp(end_date, tz='Europe/Budapest')

    if os.path.exists(f'{wind_solar_forecast_folder_path}/{filename}'):
        print(f'{wind_solar_forecast_folder_path}/{filename} exists, reading from file')
        #load
        df = pd.read_csv(f'{wind_solar_forecast_folder_path}/{filename}', index_col=0)
    else:
        print(f'{wind_solar_forecast_folder_path}/{filename} does not exist, downloading from ENTSO-E')

        #set start time to 00:00:00 and end time to 23:59:59
        start_ts = pd.Timestamp(start_date, tz='Europe/Brussels')
        end_ts = pd.Timestamp(end_date, tz='Europe/Brussels') + pd.Timedelta(days=1)

        df =client.query_wind_and_solar_forecast(country_code, start=start_ts, end=end_ts)        # Data from ENTSO-E
        
        df.to_csv(f'{wind_solar_forecast_folder_path}/{filename}')

    return df

# Create the base concatenated df

First we will create a base concatenated dataframe with all the data from the different files but only the prices

In [9]:
def get_base_prices(country_code, years=years):
    base_df_filename = get_base_df_filename(country_code, years)
    if os.path.exists(base_df_filename):
        print(f'{base_df_filename} exists, reading from file')
        df = pd.read_csv(base_df_filename, index_col=0, parse_dates=True)
    else:
        print(f'{base_df_filename} does not exist, concatenating from multiple files')

        for (start_date, end_date) in years:
            df_temp = run_querry_day_ahead_prices(country_code, start_date, end_date)

            df = df_temp if (start_date, end_date) == years[0] else pd.concat([df, df_temp])

        df.columns = [f'Price_{country_code}']
        df['Datetime'] = df.index
        df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
        df.reset_index(drop=True, inplace=True)
        #order by datetime
        df = df.sort_values(by='Datetime')
        df = df.set_index('Datetime')
        #add 1 hour to the datetime
        df.index = df.index + pd.DateOffset(hours=1)
        df.to_csv(base_df_filename)
    return df

In [10]:
def load_15min_to_hourly(df, col_name, target_col_name):
    df['Date'] = df.index.floor('h')
    df[target_col_name] = df.groupby('Date')[col_name].transform('mean')
    df2 = df[['Date', target_col_name]]
    df2 = df2.drop_duplicates()
    return df

In [11]:
def get_base_load_avg(country_code, years=years):
    base_load_df_filename = get_base_load_df_filename(country_code, years)
    if os.path.exists(base_load_df_filename):
        print(f'{base_load_df_filename} exists, reading from file')
        df = pd.read_csv(base_load_df_filename, index_col=0, parse_dates=True)
    else:
        print(f'{base_load_df_filename} does not exist, concatenating from multiple files')

        for (start_date, end_date) in years:

            df_temp = run_querry_load(country_code, start_date, end_date)

            df = df_temp if (start_date, end_date) == years[0] else pd.concat([df, df_temp])
            
        df['Datetime'] = df.index
        df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
        df.reset_index(drop=True, inplace=True)
        #order by datetime
        df = df.sort_values(by='Datetime')
        df = df.set_index('Datetime')
        #add 1 hour to the datetime
        df.index = df.index + pd.DateOffset(hours=1)
        df = load_15min_to_hourly(df, 'Actual Load', 'Load_avg')
        df = df[['Date', 'Load_avg']].drop_duplicates()
        df.drop(columns=['Date'], inplace=True)
        df.columns = [f'Load_avg_{country_code}']
        df.to_csv(base_load_df_filename)
    return df

In [12]:
def get_base_wind_solar_forecast_avg(country_code, years=years):
    base_wind_solar_forecast_df_filename = get_base_wind_solar_forecast_df_filename(country_code, years)
    if os.path.exists(base_wind_solar_forecast_df_filename):
        print(f'{base_wind_solar_forecast_df_filename} exists, reading from file')
        df = pd.read_csv(base_wind_solar_forecast_df_filename, index_col=0, parse_dates=True)
    else:
        print(f'{base_wind_solar_forecast_df_filename} does not exist, concatenating from multiple files')

        for (start_date, end_date) in years:

            df_temp = run_querry_wind_solar_forecast(country_code, start_date, end_date)

            df = df_temp if (start_date, end_date) == years[0] else pd.concat([df, df_temp])
            
        df['Datetime'] = df.index
        df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
        df.reset_index(drop=True, inplace=True)
        #order by datetime
        df = df.sort_values(by='Datetime')
        df = df.set_index('Datetime')
        #add 1 hour to the datetime
        df.index = df.index + pd.DateOffset(hours=1)
        df = load_15min_to_hourly(df, 'Solar', 'Solar_Fcast_avg')
        df = load_15min_to_hourly(df, 'Wind Onshore', 'Wind_Onshore_avg')

        df = df[['Date', 'Solar_Fcast_avg', 'Wind_Onshore_avg']].drop_duplicates()
        df.drop(columns=['Date'], inplace=True)
        df.columns = [f'Solar_Fcast_avg_{country_code}', f'Wind_Onshore_avg_{country_code}']
        df.to_csv(base_wind_solar_forecast_df_filename)
    return df

In [13]:
def get_base_df(country_codes):
    for country_code in country_codes:
        print('Starting to collect data for', country_code)
        prices_df = get_base_prices(country_code)
        load_avg_df = get_base_load_avg(country_code)
        wind_solar_forecast_df = get_base_wind_solar_forecast_avg(country_code)
        
        df = pd.concat([prices_df, load_avg_df, wind_solar_forecast_df], axis=1)
        df = df.dropna()
        df = df.reset_index()
        df = df.rename(columns={'index': 'Datetime'})
        df_final = df if country_code == country_codes[0] else pd.merge(df_final, df, on='Datetime', how='inner')

    return df_final

In [14]:
country_codes = ['HU', 'RO']
df = get_base_df(country_codes)

Starting to collect data for HU
./data/base_price_HU_('2017-01-01', '2017-12-31')_('2024-01-01', '2024-03-24').csv exists, reading from file


./data/base_load_avgs_HU_('2017-01-01', '2017-12-31')_('2024-01-01', '2024-03-24').csv exists, reading from file
./data/base_wind_solar_forecast_avgs_HU_('2017-01-01', '2017-12-31')_('2024-01-01', '2024-03-24').csv exists, reading from file
Starting to collect data for RO
./data/base_price_RO_('2017-01-01', '2017-12-31')_('2024-01-01', '2024-03-24').csv exists, reading from file
./data/base_load_avgs_RO_('2017-01-01', '2017-12-31')_('2024-01-01', '2024-03-24').csv exists, reading from file
./data/base_wind_solar_forecast_avgs_RO_('2017-01-01', '2017-12-31')_('2024-01-01', '2024-03-24').csv exists, reading from file


In [15]:
df

,Datetime,Price_HU,Load_avg_HU,Solar_Fcast_avg_HU,Wind_Onshore_avg_HU,Price_RO,Load_avg_RO,Solar_Fcast_avg_RO,Wind_Onshore_avg_RO
0,2019-01-01 00:00:00+00:00,59.90,4082.00,0.0,19.75,279.00,6110.00,0.0,88.00
1,2019-01-01 01:00:00+00:00,52.71,3985.75,0.0,27.25,245.54,5856.00,0.0,95.00
2,2019-01-01 02:00:00+00:00,36.49,3732.50,0.0,19.75,169.98,5675.00,0.0,90.00
3,2019-01-01 03:00:00+00:00,31.24,3554.25,0.0,7.25,145.52,5570.00,0.0,86.00
4,2019-01-01 04:00:00+00:00,25.98,3499.25,0.0,1.75,121.00,5524.00,0.0,83.00
...,...,...,...,...,...,...,...,...,...
45654,2024-03-24 19:00:00+00:00,85.22,5588.50,0.0,63.25,85.46,6560.25,1.0,1105.25
45655,2024-03-24 20:00:00+00:00,76.51,5461.00,0.0,65.75,76.65,6317.25,1.0,1016.75
45656,2024-03-24 21:00:00+00:00,73.74,5193.50,0.0,68.00,73.68,5877.25,1.0,928.25
45657,2024-03-24 22:00:00+00:00,70.07,4897.25,0.0,71.75,70.10,5498.75,1.0,938.75


# Create more input features

In [16]:
target_country_code = 'HU'

In [17]:
df['month'] = df['Datetime'].dt.month
df['day'] = df['Datetime'].dt.day
df['hour'] = df['Datetime'].dt.hour
df['weekday'] = df['Datetime'].dt.weekday
df['dayoftheweek'] = df['Datetime'].dt.dayofweek
df['date'] = df['Datetime'].dt.date

In [18]:
month_dummies = pd.get_dummies(df['month'], prefix='month', drop_first=True)
dayofweek_dummies = pd.get_dummies(df['dayoftheweek'], prefix='dayofweek', drop_first=True)
df = pd.concat([df, month_dummies, dayofweek_dummies], axis=1)

In [19]:
dummy_columns = month_dummies.columns.tolist() + dayofweek_dummies.columns.tolist()

In [20]:
df['Datetime-2d'] = df['Datetime'] + pd.to_timedelta(-2, unit='day')
df['Datetime-7d'] = df['Datetime'] + pd.to_timedelta(-7, unit='day')

load_map = df.set_index('Datetime')[f'Price_{target_country_code}']

df['Price-2d'] = df['Datetime-2d'].map(load_map)
df['Price-7d'] = df['Datetime-7d'].map(load_map)

In [21]:
df.isnull().sum()

Datetime                 0
Price_HU                 0
Load_avg_HU              0
Solar_Fcast_avg_HU       0
Wind_Onshore_avg_HU      0
Price_RO                 0
Load_avg_RO              0
Solar_Fcast_avg_RO       0
Wind_Onshore_avg_RO      0
month                    0
day                      0
hour                     0
weekday                  0
dayoftheweek             0
date                     0
month_2                  0
month_3                  0
month_4                  0
month_5                  0
month_6                  0
month_7                  0
month_8                  0
month_9                  0
month_10                 0
month_11                 0
month_12                 0
dayofweek_1              0
dayofweek_2              0
dayofweek_3              0
dayofweek_4              0
dayofweek_5              0
dayofweek_6              0
Datetime-2d              0
Datetime-7d              0
Price-2d               216
Price-7d               349
dtype: int64

In [22]:
df = df.dropna().reset_index(drop=True)

In [23]:
df

,Datetime,Price_HU,Load_avg_HU,Solar_Fcast_avg_HU,Wind_Onshore_avg_HU,Price_RO,Load_avg_RO,Solar_Fcast_avg_RO,Wind_Onshore_avg_RO,month,...,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,Datetime-2d,Datetime-7d,Price-2d,Price-7d
0,2019-01-08 00:00:00+00:00,51.40,5069.50,0.0,22.50,181.78,6959.00,0.0,1456.00,1,...,True,False,False,False,False,False,2019-01-06 00:00:00+00:00,2019-01-01 00:00:00+00:00,50.81,59.90
1,2019-01-08 01:00:00+00:00,42.99,4876.25,0.0,31.75,170.46,6874.00,0.0,1397.00,1,...,True,False,False,False,False,False,2019-01-06 01:00:00+00:00,2019-01-01 01:00:00+00:00,47.95,52.71
2,2019-01-08 02:00:00+00:00,41.00,4672.50,0.0,32.00,161.23,6860.00,0.0,1347.00,1,...,True,False,False,False,False,False,2019-01-06 02:00:00+00:00,2019-01-01 02:00:00+00:00,46.07,36.49
3,2019-01-08 03:00:00+00:00,39.76,4583.00,0.0,36.50,185.47,6923.00,0.0,1221.00,1,...,True,False,False,False,False,False,2019-01-06 03:00:00+00:00,2019-01-01 03:00:00+00:00,42.86,31.24
4,2019-01-08 04:00:00+00:00,42.77,4685.75,0.0,49.75,199.50,7133.00,0.0,1048.00,1,...,True,False,False,False,False,False,2019-01-06 04:00:00+00:00,2019-01-01 04:00:00+00:00,43.01,25.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45141,2024-03-24 19:00:00+00:00,85.22,5588.50,0.0,63.25,85.46,6560.25,1.0,1105.25,3,...,False,False,False,False,False,True,2024-03-22 19:00:00+00:00,2024-03-17 19:00:00+00:00,114.84,87.46
45142,2024-03-24 20:00:00+00:00,76.51,5461.00,0.0,65.75,76.65,6317.25,1.0,1016.75,3,...,False,False,False,False,False,True,2024-03-22 20:00:00+00:00,2024-03-17 20:00:00+00:00,90.77,77.89
45143,2024-03-24 21:00:00+00:00,73.74,5193.50,0.0,68.00,73.68,5877.25,1.0,928.25,3,...,False,False,False,False,False,True,2024-03-22 21:00:00+00:00,2024-03-17 21:00:00+00:00,78.11,72.33
45144,2024-03-24 22:00:00+00:00,70.07,4897.25,0.0,71.75,70.10,5498.75,1.0,938.75,3,...,False,False,False,False,False,True,2024-03-22 22:00:00+00:00,2024-03-17 22:00:00+00:00,75.69,72.10


In [24]:
from workalendar.europe import Hungary

cal = Hungary()
year_list = df['Datetime'].dt.year.unique()

In [25]:
holiday_df = pd.DataFrame()
for year in year_list:
    holidays = cal.holidays(year)
    temp_df = pd.DataFrame(holidays, columns=['date', 'holiday_name'])
    holiday_df = pd.concat([holiday_df, temp_df],
                           axis=0).reset_index(drop=True)

In [26]:
holiday_map = holiday_df.set_index('date')['holiday_name']
df['holiday_name'] = df['date'].map(holiday_map)
df['is_holiday'] = 0
df.loc[df['holiday_name'].notnull(), 'is_holiday'] = 1

df.drop(columns=['holiday_name'], inplace=True)

# Evaluation metric

In [27]:
def base_eval(y_true, y_pred):
    error = np.mean(abs(y_true - y_pred))
    print(f'Base evaluation (abs error): {error}')

In [28]:
def weighted_eval(y_true, y_pred, load):
    error = np.mean(abs((y_true - y_pred)*load))
    print(f'Weighted evaluation (abs error): {error}')

In [29]:
def do_modeling(model, train_df, test_df, input_cols, target_col):
    model.fit(train_df[input_cols], train_df[target_col])
    y_pred = model.predict(test_df[input_cols])
    y_true = test_df[target_col]
    base_eval(y_true, y_pred)
    weighted_eval(y_true, y_pred, test_df['Load_avg'])

# Testing on models

In [30]:
df.head(1).T

,0
Datetime,2019-01-08 00:00:00+00:00
Price_HU,51.4
Load_avg_HU,5069.5
Solar_Fcast_avg_HU,0.0
Wind_Onshore_avg_HU,22.5
Price_RO,181.78
Load_avg_RO,6959.0
Solar_Fcast_avg_RO,0.0
Wind_Onshore_avg_RO,1456.0
month,1


In [31]:
test_end_date = '2024-03-17'

In [32]:
in_cols = ['Price-2d', 'Price-7d', 'is_holiday', 'Solar_Fcast_avg_HU', 'Wind_Onshore_avg_HU', 'Solar_Fcast_avg_RO', 'Wind_Onshore_avg_RO'] + dummy_columns
target_col = f'Price_{target_country_code}'

## Baseline

In [33]:
def baseline_model(df):
    y_pred = df[f'Price_{target_country_code}'].shift(7*24)
    return y_pred

In [34]:
base_pred = baseline_model(df)
start_date = '2023-01-01'
end_date = '2024-03-17'
base_eval(df.loc[df['Datetime'] >= test_end_date, target_col], base_pred)
weighted_eval(df.loc[df['Datetime'] >= test_end_date, target_col], base_pred, df.loc[df['Datetime'] >= test_end_date, f'Load_avg_{target_country_code}'])

Base evaluation (abs error): 16.567239583333333
Weighted evaluation (abs error): 77618.14497395833


## ARIMA

As another base model

In [35]:
from statsmodels.tsa.arima.model import ARIMA

In [36]:
df_train = df.loc[df['Datetime'] < test_end_date]

In [37]:
#create an ARIMA model, train it on 2017-2022 and forecast 2023
model = ARIMA(df_train.loc[:,f'Price_{target_country_code}'], order=(12,1,1))
model = model.fit()

In [38]:
arima_pred = model.forecast(steps=len(df.loc[df['Datetime'] >= test_end_date]))

In [39]:
# evaluate the model
base_eval(df.loc[df['Datetime'] >= test_end_date, target_col], arima_pred)
weighted_eval(df.loc[df['Datetime'] >= test_end_date, target_col], arima_pred, df.loc[df['Datetime'] >= test_end_date, f'Load_avg_{target_country_code}'])

Base evaluation (abs error): 21.93563974563405
Weighted evaluation (abs error): 107019.05274380276


## Gradient Boosting Regressor

In [40]:
from sklearn.ensemble import GradientBoostingRegressor

In [41]:
gbr = GradientBoostingRegressor(random_state=42)

In [42]:
train = df.loc[df['Datetime'] <= test_end_date]
test = df.loc[df['Datetime'] > test_end_date]

X_train = train[in_cols]
y_train = train[target_col]
X_test = test[in_cols]
y_test = test[target_col]

In [43]:
gbr.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [44]:
gbr_pred = gbr.predict(X_test)

In [45]:
base_eval(y_test, gbr_pred)
weighted_eval(y_test, gbr_pred, test[f'Load_avg_{target_country_code}'])

Base evaluation (abs error): 14.111308200145777
Weighted evaluation (abs error): 66519.87245601627


In [46]:
# zip the feature importances with the column names and sort them from the most to the least important
feature_importances = list(zip(X_train.columns, gbr.feature_importances_))
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
feature_importances

[('Price-7d', 0.5555244047639467),
 ('Price-2d', 0.3977509042250395),
 ('month_8', 0.00812526226950278),
 ('dayofweek_1', 0.008052607465713965),
 ('dayofweek_6', 0.006987412653258569),
 ('Wind_Onshore_avg_RO', 0.005760118101848488),
 ('dayofweek_5', 0.004775398467092988),
 ('month_7', 0.002197386815899303),
 ('Wind_Onshore_avg_HU', 0.0019264497840377416),
 ('month_12', 0.001766339522601651),
 ('Solar_Fcast_avg_HU', 0.0016260745263091715),
 ('Solar_Fcast_avg_RO', 0.001618157507516575),
 ('month_10', 0.0014143739228521307),
 ('is_holiday', 0.001317480649012541),
 ('month_11', 0.0003626993247009838),
 ('dayofweek_4', 0.00023046660394003123),
 ('month_9', 0.0001662666166888682),
 ('dayofweek_2', 0.00011981537614542208),
 ('month_3', 0.00010672086019029484),
 ('dayofweek_3', 0.00010309477725665662),
 ('month_5', 6.856576644554121e-05),
 ('month_2', 0.0),
 ('month_4', 0.0),
 ('month_6', 0.0)]

# Notes

Ma 8as adatokkal 9ig a holnapit (a mait mind ismerem)
- baseline1 az 1 heti adat
- baseline2 hasonló időjárású nap

Kiértékelés:
- v1 abs hiba (hány eurót tévedtünk)
- v2 adott órában mennyi a load (termelés / fogyasztás), hiba súlyozva a teljes fogyasztással

Opciók
- recurrent nn predictor (?)
- gbm regressor
-- (előzö napi adatok, napelemek termelése, román adatok, hőmérséklet..., körny ország árai)
-- walk forward opt

keretrendszer
feture inportance alapján feature selection 
- változásuk követése !!!

(talán osztrák is számít, meg kell nézni melyik számít)

időjárási adatok (első körben tényadatok, nem előrejelzés) próbálkozni kell, drága lehet, kb kizárt 

- végén fontos és ***nem fontos*** változók listája

3 fontos időjárás (régiós, a napi bontás is jó)
- hány fok van (fűtés / hűtés)
- besugárzás
- szélerősség


-Hányszor volt negatív ár - statisztika róla (Meg tudjuk-e mondani, hogy mikor lesz negatív ár)
-- Ez is lehet célváltozó és kiértékelés

- Napi egy órát kikapcsoljuk, cél: mikor legyen (mert a többi órában többet tudunk termelni)
-- Meg lehet nézni, hogy melyik lesz a legdrágább óra
